In [ ]:
# Transformer 모델 구축 - Transformer News Summary 요약 모델
# 학습 목표 - 실무에서 사용되는 파이프라인 이해 및 적용
# - 1. 데이터셋 로드 및 json 파일 추출 
# - 2. JSON 파싱: 본문과 요약 추출 
# - 3. 데이터 전처리
# - 4. 토크나이저 적용 데이터셋 -> DataLoader 생성
# - collate_fn 적용
# - 5. 모델 정의
# - Feature Extraction + LoRA Fine-tuning 조합
# - Early Stopping 클래스 정의
# - 6. 학습 루프
# - autocast(속도 향상) 적용, GradScaler(안정적 학습) 적용
# - 7. 테스트
# - 8. 최적 모델 로드
# - 9. 실제 요약 생성
# - 10. ROUGE의 주요 지표
# - 11. FastAPI 추론 서비스
# - /llm_app/transformer_summary_news_19_app.py
# - FastAPI 구동: 터미널에서 구동, uvicorn transformer_summary_news_19_app:app --reload
# - 윈도우 파워쉘: Invoke-RestMethod -Uri "http://127.0.0.1:8000/summarize" -Method Post -ContentType "application/json" -Body '{"text":"I really love this movie, it was fantastic!"}'
# - Postman app
# - API 코드로 테스트: Python, Java...

In [2]:
# BartTokenizer/BartForConditionalGeneration에서 사용된 facebook/bart-large-cnn 모델
# - 인코더,디코더 구조를 가진 Seq2Seq 모델
# - 인코더: 입력 문장을 양방향으로 인코딩
# - 디코더: 요약이나 번역 같은 출력 문장을 생성
# - CNN/DailyMail 뉴스 요약 데이터셋으로 파인튜닝된 버전, 요약 태스크에 최적화된 사전학습 모델

# 1. 입력 토큰화(DataLoader 배치): 
# - input_ids:(batch_size,seq_len)->(8,1024)
# - attention_mask:(8,1024)
# - labels:(8,128) 요약 문장 토큰 ID

# 2. 인코더 출력
# - 인코더는 입력을 hidden state로 변환
# - shape: (batch_size,seq_len,hidden_dim), (8,1024,1024)

# 3. 디코더 입력
# - 디코더는 이전까지 생성된 토큰을 받아 다음 토큰을 예측
# - shape: (batch_size,tgt_seq_len,hidden_dim), (8,128,1024)

# 4. 출력 로짓
# - 디코더 마지막 레이어에서 vocab 크기만큼 확률 분포 출력
# - shape: (batch_size,tgt_seq_len,vocab_size), (8,128,50265)

# 5. 디코딩
# - model.generate()는 Beam Search 등을 통해 최종 토큰 시퀀스를 선택
# - 최종 요약 shape: (batch_size,tgt_seq_len), (8,128)

# PeftModelForSeq2SeqLM(
#   (base_model): LoraModel(
#     (model): BartForConditionalGeneration(
#       (model): BartModel(
#         - 입력 (batch_size,seq_len) (8,1024)
#         - 출력 (batch_size,seq_len,hiddem) (8,1024,1024)
#         (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
#         (encoder): BartEncoder(
#         - 입력 (8,1024,1024) 
#         - 각 레이어에서 Self-Attention->FeedForward->LayerNorm를 거친다
#         - 출력 (8,1024,1024) hidden_dim 유지
#           (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
#           (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
#           (layers): ModuleList(
#             (0-11): 12 x BartEncoderLayer(
#               (self_attn): BartAttention(
#                 (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
#                 (v_proj): lora.Linear(
#                   (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
#                   (lora_dropout): ModuleDict(
#                     (default): Dropout(p=0.1, inplace=False)
#                   )
#                   (lora_A): ModuleDict(
#                     (default): Linear(in_features=1024, out_features=16, bias=False)
#                   )
#                   (lora_B): ModuleDict(
#                     (default): Linear(in_features=16, out_features=1024, bias=False)
#                   )
#                   (lora_embedding_A): ParameterDict()
#                   (lora_embedding_B): ParameterDict()
#                   (lora_magnitude_vector): ModuleDict()
#                 )
#                 (q_proj): lora.Linear(
#                   (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
#                   (lora_dropout): ModuleDict(
#                     (default): Dropout(p=0.1, inplace=False)
#                   )
#                   (lora_A): ModuleDict(
#                     (default): Linear(in_features=1024, out_features=16, bias=False)
#                   )
#                   (lora_B): ModuleDict(
#                     (default): Linear(in_features=16, out_features=1024, bias=False)
#                   )
#                   (lora_embedding_A): ParameterDict()
#                   (lora_embedding_B): ParameterDict()
#                   (lora_magnitude_vector): ModuleDict()
#                 )
#                 (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
#               )
#               (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
#               (activation_fn): GELUActivation()
#               (fc1): Linear(in_features=1024, out_features=4096, bias=True)
#               (fc2): Linear(in_features=4096, out_features=1024, bias=True)
#               (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
#             )
#           )
#           (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
#         )
#         (decoder): BartDecoder(
#         - 입력 (batch_size,tgt_seq_len) (8,128)
#         - 임베딩 후 (8,128,1024)
#         - 각 레이어에서 Self-Attention + Encoder-Decoder Attention->FeedForward 수행
#         - 출력 (8,128,1024)
#           (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
#           (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
#           (layers): ModuleList(
#             (0-11): 12 x BartDecoderLayer(
#               (self_attn): BartAttention(
#                 (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
#                 (v_proj): lora.Linear(
#                   (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
#                   (lora_dropout): ModuleDict(
#                     (default): Dropout(p=0.1, inplace=False)
#                   )
#                   (lora_A): ModuleDict(
#                     (default): Linear(in_features=1024, out_features=16, bias=False)
#                   )
#                   (lora_B): ModuleDict(
#                     (default): Linear(in_features=16, out_features=1024, bias=False)
#                   )
#                   (lora_embedding_A): ParameterDict()
#                   (lora_embedding_B): ParameterDict()
#                   (lora_magnitude_vector): ModuleDict()
#                 )
#                 (q_proj): lora.Linear(
#                   (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
#                   (lora_dropout): ModuleDict(
#                     (default): Dropout(p=0.1, inplace=False)
#                   )
#                   (lora_A): ModuleDict(
#                     (default): Linear(in_features=1024, out_features=16, bias=False)
#                   )
#                   (lora_B): ModuleDict(
#                     (default): Linear(in_features=16, out_features=1024, bias=False)
#                   )
#                   (lora_embedding_A): ParameterDict()
#                   (lora_embedding_B): ParameterDict()
#                   (lora_magnitude_vector): ModuleDict()
#                 )
#                 (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
#               )
#               (activation_fn): GELUActivation()
#               (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
#               (encoder_attn): BartAttention(
#                 (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
#                 (v_proj): lora.Linear(
#                   (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
#                   (lora_dropout): ModuleDict(
#                     (default): Dropout(p=0.1, inplace=False)
#                   )
#                   (lora_A): ModuleDict(
#                     (default): Linear(in_features=1024, out_features=16, bias=False)
#                   )
#                   (lora_B): ModuleDict(
#                     (default): Linear(in_features=16, out_features=1024, bias=False)
#                   )
#                   (lora_embedding_A): ParameterDict()
#                   (lora_embedding_B): ParameterDict()
#                   (lora_magnitude_vector): ModuleDict()
#                 )
#                 (q_proj): lora.Linear(
#                   (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
#                   (lora_dropout): ModuleDict(
#                     (default): Dropout(p=0.1, inplace=False)
#                   )
#                   (lora_A): ModuleDict(
#                     (default): Linear(in_features=1024, out_features=16, bias=False)
#                   )
#                   (lora_B): ModuleDict(
#                     (default): Linear(in_features=16, out_features=1024, bias=False)
#                   )
#                   (lora_embedding_A): ParameterDict()
#                   (lora_embedding_B): ParameterDict()
#                   (lora_magnitude_vector): ModuleDict()
#                 )
#                 (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
#               )
#               (encoder_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
#               (fc1): Linear(in_features=1024, out_features=4096, bias=True)
#               (fc2): Linear(in_features=4096, out_features=1024, bias=True)
#               (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
#             )
#           )
#           (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
#         )
#       )
#       - Linear(in_features=1024, out_features=50264)
#       - 입력 (8,128,1024)
#       - 출력 (8,128,50264) 50264는 vocab size-> 각 위치에서 50264 형태의 최종 토큰 시퀀스 선택
#       - 최종 토큰 시퀀스 선택 (8,128)
#       (lm_head): Linear(in_features=1024, out_features=50264, bias=False)
#     )
#   )
# )

In [3]:
# 데이터셋 로드 및 json 파일 추출
import os
import json

data_dir = './llm_data/ai_hub_news_summary' # 데이터셋 경로

all_data = []

for file_name in os.listdir(data_dir): # 파일명 추출
    if file_name.endswith('.json'): # .json 파일 추출
        with open(os.path.join(data_dir, file_name), 'r', encoding='utf-8') as f: # 파일 일기
            data = json.load(f) # json 파일 로드
            all_data.append(data)

In [4]:
# JSON 파싱: 본문과 요약 추출
dataset = [] # 최종적으로 본문과 요약문을 담는 리스트

for item in all_data: # JSON 데이터셋    
    passage = item['Meta(Refine)']['passage'] # 본문

    summaries = [] # 요약문
    if item['Annotation'].get('summary1'):
        summaries.append(item['Annotation'].get('summary1'))
    if item['Annotation'].get('summary2'):        
        summaries.append(item['Annotation'].get('summary2'))

    # 요약이 없는 경우 제외
    if passage and summaries:
        dataset.append({'text': passage, 'summary': summaries})

In [5]:
# 데이터 전처리
def clean_text(text):
    # 줄바꿈 -> 공백으로 바꾼다, 문장열 앞뒤 불필요한 공백 제거
    text = text.replace('\n', ' ').strip()
    return text

for sample in dataset:
    sample['text'] = clean_text(sample['text']) # 문자열
    sample['summary'] = [ clean_text(s) for s in sample['summary'] ] # 리스트

# 데이터 확인
print(len(dataset), dataset[0])

10800 {'text': '보수진영 사분오열 속 ‘국민통합연대’ 띄운 비박계 크리스마스를 앞둔 지난 23일 오전 서울 프레스센터 국제회의실.   보수분열 극복을 내건 ‘국민통합연대’가 창립대회를 열었다.   햇살 없이 착 가라앉은 날씨에 동지 바람이 매서웠지만 행사장 안은 달아올랐다.   문재인 정권을 향한 맹폭격이 이어졌고 ‘무능, 기만의 오만방자한 정권에 사망을 선고한다’는 창립선언문이 나왔다.   홍준표 전 자유한국당 대표 등 전현직 의원 20여명을 포함해 500여명이 자리를 빼곡하게 메웠다.   총선이 불과 석달 남짓이다.   야권 인사들이 정권을 두들겨 패는 거야 이상한 일이 아니다.   눈 여겨 볼 대목은 모인 사람이 대부분 친이·비박계(친이명박·비박근혜) 인사들이란 점이다.   박관용 전 국회의장, 이문열 작가와 함께 보수쪽 명망가 여럿이 이름을 올리고 더러 참석했다.   전광훈 목사는 축사를 했다.   그래도 이명박 정권서 요직을 맡았던 사람들이 주축이다.   이재오 중앙집행위원장과 홍준표 전 대표가 한가운데 있다.   두 사람은 ‘친박 그룹’에 둘러싸인 황교안 대표와 한국당에 불편한 기색을 감추지 않는 중이다.   홍 전 대표는 이튿날 “무기력한 야당만 믿고 따르기엔 너무 답답하고 앞날이 보이지 않아 창립한 게 국민통합연대”란 글을 올렸다.   31일엔 “한국당 지도부는 총사퇴하고 비상대책위를 꾸려야 한다”고 황 대표 사퇴를 요구했다.   이재오 위원장은 지난 10월3일 광화문의 조국 규탄집회장에서 “자유한국당은 집회에서 빠지라”고 외쳤다.   가뜩이나 뿔뿔이 흩어진 각자도생의 보수세력이다.   한국당과 우리공화당에다 새로운 보수당, 이언주 신당, 이정현 신당이 나올 판이다.   게다가 개정선거법의 준연동형 비례대표제는 군소정당에 유리한 분열요인이다.   중앙선관위에 등록된 정당이 34개인데 창당준비위원회를 설립한 예비정당만 16개에 달한다.   야권 빅텐트를 외칠만한 상황이긴 하다.   그런데 통합을 내건 이재오 위원장은 “어느 한 정당이나

In [6]:
# 토크나이저 적용 데이터셋 -> DataLoader 생성, collate_fn 적용
import torch
from transformers import BartTokenizer
from torch.utils.data import DataLoader, random_split

# BART 토크라이저 로드, Bart vocab size: 50265
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
# print('Bart vocab size:', tokenizer.vocab_size)

# collate_fn 정의
def collate_fn(batch):
    texts = [ item['text'] for item in batch ]
    summaries = [ ' '.join(item['summary']) for item in batch ]

    # 입력과 요약을 토큰화
    inputs = tokenizer(
        texts, # 본문 리스트
        max_length=1024, # BART 입력 최대 길이
        truncation=True,
        padding='max_length', # padding='max_length' max_length까지 모든 문장을 강제 패딩, padding=True 배치 내에서 가장 긴 문장 길이에 맞추어 패딩
        return_tensors='pt'
    )
    labels = tokenizer(
        summaries, # 요약 리스트를 하나의 문자열로 합치기
        max_length=128, # 요약 최대 길이
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': labels['input_ids']        
    }
dataset_small = dataset[:2000] # 데이터 축소
total_size = len(dataset_small) # 전체 데이터 길이
train_size = int(0.8 * total_size) # 비율 설정(예시: train 80%, val 10%, test 10%)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size
generator = torch.Generator().manual_seed(42) # 시드 고정
train_dataset, val_dataset, test_dataset = random_split(dataset_small, [train_size, val_size, test_size], generator=generator) # 데이터 분리

# DataLoader 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=collate_fn
)
val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn
)

# 배치 확인
batch = next(iter(train_loader))
print(batch['input_ids'].shape, batch['labels'].shape)

torch.Size([8, 1024]) torch.Size([8, 128])


In [7]:
# 모델 정의
# - Feature Extraction + LoRA Fine-tuning 조합

import torch
from transformers import BartForConditionalGeneration
from peft import LoraConfig, get_peft_model
from torch.amp import autocast, GradScaler
import os

# GPU 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'PyTorch Version: {torch.__version__}, Device: {device}')

# BartForConditionalGeneration 베이스 모델
model = BartForConditionalGeneration.from_pretrained(model_name)

# Feature Extraction
for name, param in model.named_parameters():
    if 'lora' not in name: # LoRA 모듈이 아닌 경우
        param.requires_grad = False # 모델 본체 동결

# LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias='none',
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, lora_config)
model.to(device)

# 모델 확인, trainable params: 2,359,296 || all params: 408,649,728 || trainable%: 0.5773
model.print_trainable_parameters() # LoRA 적용 확인
print(model)

# 최적화 설정
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scaler = GradScaler()

# 반복횟수
num_epochs = 3

# Early Stopping 클래스 정의
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.0, path='./llm_models/19_transformer_summary_news/best_model.pt'):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
        self.path = path
    
    def __call__(self, valid_loss, model):
        if self.best_loss is None:
            self.best_loss = valid_loss
            self.save_checkpoint(model)
        # 성능 개선 -> 최적 모델 갱신
        elif valid_loss < self.best_loss - self.min_delta:
            self.best_loss = valid_loss
            self.counter = 0
            self.save_checkpoint(model)
        # 개선 없음
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
    
    def save_checkpoint(self, model):
        # 디렉토리만 생성
        folder = os.path.dirname(self.path)
        if folder !='' and not os.path.exists(folder):
            os.makedirs(folder)
        
        torch.save(model.state_dict(), self.path)
        print(f'Best model saved at {self.path}')
early_stopping = EarlyStopping(patience=3, min_delta=0.001)

PyTorch Version: 2.8.0+cu129, Device: cuda
trainable params: 2,359,296 || all params: 408,649,728 || trainable%: 0.5773
PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): BartForConditionalGeneration(
      (model): BartModel(
        (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
        (encoder): BartEncoder(
          (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
          (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
          (layers): ModuleList(
            (0-11): 12 x BartEncoderLayer(
              (self_attn): BartAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                  

In [8]:
# 학습 루프: autocast(속도 향상) 적용, GradScaler(안정적 학습) 적용
# autocast 적용: 연산을 FP16(half precision)과 FP32(full precision)중 적절히 선택해서 실행
# - 속도 향상: 대부분의 연산을 FP16으로 처리해 GPU 연산 속도를 높인다
# - 안정성 유지: 손실이 큰 연산(예시:소프트맥스,레이어정규화)은 FP32로 자동 변환해 정확도를 보장한다
# GradScaler 적용: FP16 학습에서는 작은 값이 underflow(0으로 사라짐)될 위험이 있다
# - 안정적 학습 보장: GradScaler는 손실(loss)를 크게 스케일링해서 역전파 시 그래디언트가 사라지지 않도록 한다
# - 이후 업데이트 단계에서 다시 원래 크기로 되돌려 안정적인 학습을 보장한다. 즉 FP16 학습에서 발생할 수 있는 수치 불안정 문제를 해결하는 역할

from tqdm import tqdm

for epoch in range(num_epochs):
    # train
    model.train() # 학습 모드 지정
    total_loss = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1} [Train]'):
        optimizer.zero_grad() # 오차역전파 코드, 미분 전 가중치/바이어스 파라미터 초기화

        # 학습데이터 GPU 지정
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # AMP(Automatic Mixed Precision) GPU에서 연산 속도와 메모리 효율 향상
        with autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels) # 모델 예측
            loss = outputs.loss # 손실값
            logits = outputs.logits # Softmax 계산된 logits값
        
        # 오차역전파(GradScaler로 안정성 확보)
        scaler.scale(loss).backward() # 미분연산
        scaler.step(optimizer) # 미분 연산 후 가중치/바이어스 파라미터 업데이트
        scaler.update()

        total_loss += loss.item() # 손실 누적
    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f}')

    # validation
    model.eval() # 검증/추론 모드 지정
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f'Epoch {epoch+1} [Validation]'):
            # 검증데이터 GPU 지정
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # AMP(Automatic Mixed Precision) GPU에서 연산 속도와 메모리 효율 향상
            with autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels) # 모델 예측
                loss = outputs.loss # 손실값
                # logits = outputs.logits # Softmax 계산된 logits값
            
                val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    # Early Stopping 객체 호출
    early_stopping(val_loss, model)
    status = 'STOP' if early_stopping.early_stop else 'CONTINUE' # # Early Stopping status 상태값

    print(f'Epoch {epoch+1} | Validation Loss: {avg_val_loss:.4f}')
    
    # Early Stopping 체크
    if early_stopping.early_stop: # early_stop=True 학습 종료
        print('Early stopping triggered')
        break 

Epoch 1 [Train]: 100%|██████████| 200/200 [50:27<00:00, 15.14s/it]


Epoch 1 | Train Loss: 1.3488


Epoch 1 [Validation]: 100%|██████████| 25/25 [02:27<00:00,  5.90s/it]


Best model saved at ./llm_models/19_transformer_summary_news/best_model.pt
Epoch 1 | Validation Loss: 0.9965


Epoch 2 [Train]: 100%|██████████| 200/200 [50:49<00:00, 15.25s/it]


...

Best model saved at ./llm_models/19_transformer_summary_news/best_model.pt
Epoch 2 | Validation Loss: 0.9311


Epoch 3 [Train]: 100%|██████████| 200/200 [51:17<00:00, 15.39s/it]


Epoch 3 | Train Loss: 0.9553


Epoch 3 [Validation]: 100%|██████████| 25/25 [02:16<00:00,  5.45s/it]


Best model saved at ./llm_models/19_transformer_summary_news/best_model.pt
Epoch 3 | Validation Loss: 0.9057


In [9]:
# 테스트
model.eval() # 검증/추론 모드 지정
test_loss = 0

with torch.no_grad(): # 미분 연산 하지 않음
    for batch in tqdm(test_loader, desc='Test'): # 테스트 데이터 GPU 지정
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # AMP(Automatic Mixed Precision) GPU에서 연산 속도와 메모리 효율 향상
        with autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss        
        test_loss += loss.item() # 손실값 누적    
avg_test_loss = test_loss / len(test_loader) # 평균 손실값 게산
print(f'Final Test Loss: {avg_test_loss:.4f}')

Test: 100%|██████████| 25/25 [01:43<00:00,  4.15s/it]

Final Test Loss: 0.9345


In [7]:
# 최적 모델 로드
import torch
from transformers import BartForConditionalGeneration
from peft import LoraConfig, get_peft_model

# 모델 로드
model_name = 'facebook/bart-large-cnn'
base_model = BartForConditionalGeneration.from_pretrained(model_name)

# LoRA 설정, 추론시 LoRA 설정이 우선 적용되어야 가중치/바이어스 키 에러가 발생 하지 않는다
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias='none',
    task_type='SEQ_2_SEQ_LM'
)
model = get_peft_model(base_model, lora_config)

# torch.load() 파일에서 파라미터(가중치) 딕셔너리를 불러옴
# model.load_state_dict() 불러온 파리미터를 모델 구조에 맞게 적용
model.load_state_dict(torch.load('./llm_models/19_transformer_summary_news/best_model.pt'))

# GPU 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device) # 모델을 실행할 디바이스(GPU or CPU)에 올린다
# 검증/추론 모드 전환 
# - model.eval() 검증/추촌 모드 에서는 Dropout 등이 비활성화되어 일관된 추론 결과를 보장한다, 
# - model.train() 학습 모드 에서는 Dropout, BatchNorm 등이 활성화되어 파라미터 업데이트를 준비 한다
model.eval() 

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): BartForConditionalGeneration(
      (model): BartModel(
        (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
        (encoder): BartEncoder(
          (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
          (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
          (layers): ModuleList(
            (0-11): 12 x BartEncoderLayer(
              (self_attn): BartAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_features=16, bias=False)
                  )
                  (lora_B): Modul

In [8]:
# 실제 요약 생성
sample = test_dataset[0] # 테스트 데이터셋에서 첫번째 샘플
text = sample['text'] # 기사 본문 추출

# 본문을 토크나이저로 인코딩 -> GPU로 이동
inputs = tokenizer( 
    text, # 본문(text)을 BART 모델이 이해할 수 있는 토큰ID(input_ids)와 어텐션 마스크(attention_mask)로 변환
    max_length=1024, # 입력 문장을 최대 1024 토큰까지 자름
    truncation=True,
    padding='max_length', # 모든 문장을 동일 길이(1024)로 맞추기 위해 패딩 추가
    return_tensors='pt'
).to(device)

# 학습된 모델로 요약 생성
summary_ids = model.generate(
    inputs=inputs['input_ids'], # input_ids, attention_mask를 모델에 넣어 요약 생성
    attention_mask=inputs['attention_mask'],
    max_length=128, # 요약 최대 길이
    num_beams=4, # Beam Search로 다양한 후보 탐색
    early_stopping=True # EOS 토큰 나오면 조기 종료
)

# 결과 디코딩, 모델이 생성한 토큰 ID를 사람이 읽을 수 있는 문자열로 변환, <pad>/<eos> 같은 특수 토큰 제거
# print('sample[text]:', sample['text'])
print('Generated Summary:', tokenizer.decode(summary_ids[0], skip_special_tokens=True))
print('Reference Summary:', ' '.join(sample['summary']))

Generated Summary: 정부가 3월 개강을 앞두고 본격적으로 입국하는 유학생을 관리한 예산 지출안을 메우기 위해 용도
Reference Summary: 기획재정부는 신종 코로나바이러스 감염증 확산에 대비해 개강을 앞두고 집중적으로 입국하는 중국인 유학생을 관리할 수 있도록 예비비 42억 원의 지출안을 국무회의에서 의결했다. 기획재정부는 신종 코로나바이러스 감염증(코로나 19) 확산에 대비해 중국에서 입국하는 유학생을 관리할 목적의 예비비 42억원 지출안을 국무회의에서 의결했다고 25일 밝혔다. 박호성 기재부 교육예산과장은 “대학과 중앙정부ㆍ지방자치단체의 유기적 연계를 통해 이번 주부터 집중적으로 입국하는 중국 유학생을 관리할 수 있도록 현장의 부족한 인력 확보 및 방역 물품을 국고로 지원하기로 했다”고 설명했다. 정부가 3월 개강을 앞두고 본격적으로 입국할 중국인 유학생을 관리하는 데 예산을 푼다.


In [ ]:
# ROUGE의 주요 지표
# - Precision: 모델 요약 중 참조 요약과 겹치는 비율
# - Recall: 참조 요약 중 모델 요약과 겹치는 비율
# - F1-score: Precision과 Recall의 조화 평균
# ROUGE-1
# - 단어(uni-gram) 단위로 참조 요약과 생성 요약의 겹침 정도를 측정
# - 핵심 키워드를 잘 잡았는지 확인 가능
# ROUGE-2
# - 2-gram(연속된 두 단어) 단위 겹침 정도
# - 문맥과 표현의 자연스러움 평가에 유용
# ROUGE-L
# - Longest Common Subsequence(최장 공통 부분열) 기반
# - 문장 구조와 흐름이 얼마나 비슷한지 확인 가능

import evaluate
from tqdm import tqdm

rouge = evaluate.load('rouge') # ROUGE 로드

model.eval() # 검증/추론 모드 적용
with torch.no_grad():
    for batch in tqdm(test_loader, desc='ROUGE Evaluation'):
        # 테스트 데이터 로드 -> GPU 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 요약 생성
        summary_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=128,
            num_beams=4,
            early_stopping=True
        )

        # 디코딩
        generated_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
        reference_summaries = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # ROUGE 업데이트
        rouge.add_batch(predictions=generated_summaries, references=reference_summaries)

# 최종 점수 계산
results = rouge.compute() 
print('ROUGE Scores:', results)

ROUGE Evaluation: 100%|██████████| 25/25 [26:15<00:00, 63.01s/it]

ROUGE Scores: {'rouge1': 0.10679761904761903, 'rouge2': 0.002, 'rougeL': 0.1078452380952381, 'rougeLsum': 0.1074047619047619}


In [6]:
# 문장 추론
import requests

url = "http://127.0.0.1:8000/summarize"
data = {
    "text": "오늘은 날씨가 맑고 따뜻해서 야외 활동하기 좋은 날이다. 하지만 오후에는 비가 올 가능성이 있어 우산을 챙기는 것이 좋다.", 
    "max_length": 50, 
    "num_beams": 4
}

response = requests.post(url=url, json=data)
print(response.status_code)
print(data["text"]) # 요청 원문
print(response.text) # 원본 응답 확인
# print(response.json())

200
오늘은 날씨가 맑고 따뜻해서 야외 활동하기 좋은 날이다. 하지만 오후에는 비가 올 가능성이 있어 우산을 챙기는 것이 좋다.
{"summary":"오늘은 날씨가 맑고 따뜻해서 야외 활동"}
